In [1]:
import pandas as pd 
from optbinning import OptimalBinning
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('gen_data.csv')

### Tính đầy đủ 

In [3]:
pd.set_option('display.max_rows', None)

Tỷ lệ missing theo cột

In [4]:
missing_col = pd.DataFrame({
    'missing_count': df.isna().sum(),
    'missing_pct': df.isna().mean() * 100
}).sort_values('missing_pct', ascending=False)

missing_col

,missing_count,missing_pct
SOCIF,0,0.0
N_AVG_DD_12M,0,0.0
FLAG_SALARY_ACC,0,0.0
FLAG_DEPOSIT,0,0.0
CBAL_SHORTTERM_LOAN,0,0.0
CBAL_LONGTERM_LOAN,0,0.0
HAS_SHORTTERM_LOAN,0,0.0
HAS_LONGTERM_LOAN,0,0.0
DURATION_MAX,0,0.0
REMAINING_DURATION_MAX,0,0.0


In [5]:
pd.set_option('display.max_rows', 20)

Tỷ lệ missing theo row

In [6]:
df['ROW_MISSING_PCT'] = df.isna().mean(axis=1) * 100

df['ROW_MISSING_PCT'].describe()

count    1138159.0
mean           0.0
std            0.0
min            0.0
25%            0.0
50%            0.0
75%            0.0
max            0.0
Name: ROW_MISSING_PCT, dtype: float64

In [7]:
df['FLAG_HIGH_MISSING'] = (df['ROW_MISSING_PCT'] > 40).astype(int)

### Tính duy nhất 

Trùng khóa logic (SOCIF – year)

In [8]:
dup_key = df.duplicated(subset=['SOCIF', 'year'], keep=False)

df.loc[dup_key].shape

(0, 55)

Trùng toàn bộ bản ghi (record duplicate)

In [9]:
dup_full = df.duplicated(keep=False)

dup_full.sum()


np.int64(0)

In [10]:
dup_rate = dup_full.mean() * 100
print(f"Tỷ lệ bản ghi trùng lặp: {dup_rate:.2f}%")


Tỷ lệ bản ghi trùng lặp: 0.00%


### Tính kịp thời 

Check dùng future information (leakage)

In [11]:
# BAD_NEXT_12M chỉ được missing ở năm cuối
leak_check = df[
    (df['year'] < df['year'].max()) &
    (df['BAD_NEXT_12M'].isna())
]

leak_check.shape


(0, 55)

Check continuity theo năm

In [12]:
year_count = df.groupby('year')['SOCIF'].nunique()
year_count


year
2018    199938
2019    204410
2020    156132
2021    160135
2022    206641
2023    210903
Name: SOCIF, dtype: int64

### Tính phù hợp 

In [13]:
rule_valid_age = (
    df['TUOI'].between(0, 120)
)

df.loc[~rule_valid_age].shape


(0, 55)

In [14]:
rule_valid_ltv = df['LTV'].between(0, 400)

df.loc[~rule_valid_ltv].shape


(0, 55)

In [15]:
rule_valid_duration = (
    (df['DURATION_MAX'] >= 0) &
    (df['REMAINING_DURATION_MAX'] >= 0)
)

df.loc[~rule_valid_duration].shape


(0, 55)

In [16]:
valid_gender = df['C_GIOITINH'].isin(['M', 'F', 'O'])
df.loc[~valid_gender].shape

(0, 55)

In [17]:
df['FLAG_INVALID'] = ~(
    rule_valid_age &
    rule_valid_ltv &
    rule_valid_duration &
    valid_gender
)

In [18]:
pd.set_option('display.max_columns', None)

In [19]:
df.describe()

,SOCIF,TRINHDO,SOHUUNHA,NHANVIENBIDV,INHERENT_RISK,REF_MONTH,REF_DAY,year,BASE_AUM,CURRENT_RISK,TUOI,INCOME,CBAL,CBALORG,AFLIMT_MAX,AFLIMT_MIN,AFLIMT_AVG,CBAL_AVG,CBAL_MAX,CBAL_MIN,COLLATERAL_VALUE,LTV,N_AVG_DEPOSIT_12M,N_AVG_DEPOSIT_6M,N_AVG_DD_12M,N_AVG_CD_12M,FLAG_SALARY_ACC,FLAG_DEPOSIT,CBAL_SHORTTERM_LOAN,CBAL_LONGTERM_LOAN,HAS_SHORTTERM_LOAN,HAS_LONGTERM_LOAN,DURATION_MAX,REMAINING_DURATION_MAX,TIME_TO_OP_MAX,RATE_AVG,PURCOD_MAX,PURCOD_MIN,MAX_DPD_12M,MAX_DPD_12M_OBS,AVG_OD_DPD_12M,SUM_ALL_OD_12M,BAD_CURRENT,XULYNO,MAX_NHOMNOCIC,N_AVG_OVERDUE_CBAL_12M,CBAL_TO_INC_12MON,REAL_GDP_GROWTH_12M,BAD_NEXT_12M,ROW_MISSING_PCT,FLAG_HIGH_MISSING
count,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1138159.0,1138159.0,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1138159.0,1138159.0,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1.138159e+06,1138159.0,1138159.0
mean,1.200010e+06,1.962980e+00,3.701521e-01,1.017520e-02,-4.510839e-01,6.503928e+00,1.451262e+01,2.020529e+03,1.556347e+07,-4.226918e-01,3.872472e+01,2.464471e+07,3.565615e+08,3.922438e+08,7.711565e+08,6.940409e+08,7.325987e+08,3.564648e+08,4.283552e+08,2.851719e+08,7.527551e+08,1.326504e+02,5.593652e+06,5.593652e+06,1.118730e+06,4.474922e+06,9.130236e-01,8.283632e-01,1.780942e+08,1.784673e+08,1.0,1.0,9.134755e+01,4.747696e+01,4.387059e+01,8.209320e+00,4.599582e+00,4.599582e+00,2.444912e+01,2.444912e+01,6.868355e+00,4.841773e+01,0.0,0.0,1.618010e+00,1.664609e+08,1.442311e+01,5.658553e+00,8.424306e-02,0.0,0.0
std,1.153839e+05,8.243087e-01,4.828455e-01,1.003578e-01,6.123859e-01,3.452058e+00,8.088168e+00,1.771243e+00,2.285007e+07,7.559220e-01,9.699934e+00,1.034298e+07,2.536160e+08,2.801574e+08,3.580181e+08,3.222163e+08,3.401172e+08,2.583863e+08,3.102669e+08,2.067090e+08,9.677215e+08,8.800866e+01,9.936494e+06,9.936494e+06,1.987299e+06,7.949195e+06,2.818006e-01,3.770647e-01,1.785742e+08,1.791599e+08,0.0,0.0,6.662703e+01,4.469349e+01,4.207309e+01,2.019898e+00,2.538326e+00,2.538326e+00,2.408371e+01,2.408371e+01,8.200050e+00,5.201602e+01,0.0,0.0,4.858742e-01,1.306047e+08,7.527059e+00,2.034374e+00,2.777521e-01,0.0,0.0
min,1.000000e+06,1.000000e+00,0.000000e+00,0.000000e+00,-1.414093e+00,1.000000e+00,1.000000e+00,2.018000e+03,4.286500e+04,-3.417804e+00,1.800000e+01,9.490482e+06,2.032390e+05,2.180770e+05,2.447903e+08,2.203112e+08,2.325508e+08,2.414150e+05,2.896980e+05,1.931320e+05,3.429200e+04,1.015334e-02,9.742000e+03,9.742000e+03,1.948000e+03,7.794000e+03,0.000000e+00,0.000000e+00,3.000000e+00,1.400000e+01,1.0,1.0,6.000000e+00,0.000000e+00,1.000000e+00,5.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,1.000000e+00,0.000000e+00,1.559263e-02,2.580000e+00,0.000000e+00,0.0,0.0
25%,1.100335e+06,1.000000e+00,0.000000e+00,0.000000e+00,-9.131795e-01,4.000000e+00,7.000000e+00,2.019000e+03,4.347134e+06,-9.459798e-01,3.200000e+01,1.814194e+07,1.874278e+08,2.056226e+08,5.464265e+08,4.917838e+08,5.191052e+08,1.849322e+08,2.223865e+08,1.479458e+08,4.774380e+06,2.023715e+01,1.350567e+06,1.350567e+06,2.701130e+05,1.080454e+06,1.000000e+00,1.000000e+00,5.875968e+07,5.882350e+07,1.0,1.0,3.600000e+01,1.700000e+01,1.500000e+01,6.701521e+00,2.000000e+00,2.000000e+00,5.000000e+00,5.000000e+00,1.000000e+00,8.000000e+00,0.0,0.0,1.000000e+00,8.304166e+07,8.748897e+00,2.910000e+00,0.000000e+00,0.0,0.0
50%,1.200003e+06,2.000000e+00,0.000000e+00,0.000000e+00,-5.662085e-01,7.000000e+00,1.500000e+01,2.021000e+03,8.779688e+06,-4.684530e-01,3.900000e+01,2.216247e+07,2.934728e+08,3.223247e+08,6.826072e+08,6.143465e+08,6.484769e+08,2.911936e+08,3

In [20]:
pd.set_option('display.max_columns', 20)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138159 entries, 0 to 1138158
Data columns (total 56 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   SOCIF                   1138159 non-null  int64  
 1   C_GIOITINH              1138159 non-null  object 
 2   TRINHDO                 1138159 non-null  int64  
 3   TTHONNHAN               1138159 non-null  object 
 4   SOHUUNHA                1138159 non-null  int64  
 5   NHANVIENBIDV            1138159 non-null  int64  
 6   INHERENT_RISK           1138159 non-null  float64
 7   REF_MONTH               1138159 non-null  int64  
 8   REF_DAY                 1138159 non-null  int64  
 9   year                    1138159 non-null  int64  
 10  BASE_AUM                1138159 non-null  int64  
 11  CURRENT_RISK            1138159 non-null  float64
 12  TUOI                    1138159 non-null  int64  
 13  SNAPSHOT_DATE           1138159 non-null  object 
 14  IN

In [22]:
cat_cols = df.select_dtypes(include=['object', 'category']).columns

for col in cat_cols:
    print(f"\n{'='*20} {col} {'='*20}")
    vc = df[col].value_counts(dropna=False)
    pct = df[col].value_counts(normalize=True, dropna=False) * 100
    print(pd.concat([vc, pct.rename('pct_%')], axis=1))


==================== C_GIOITINH ====================
             count      pct_%
C_GIOITINH                   
F           559790  49.183814
M           555844  48.837113
O            22525   1.979073

==================== TTHONNHAN ====================
            count      pct_%
TTHONNHAN                   
Married    595719  52.340578
Single     542440  47.659422

==================== SNAPSHOT_DATE ====================
               count     pct_%
SNAPSHOT_DATE                 
2023-12-08       688  0.060448
2023-10-11       688  0.060448
2023-05-12       688  0.060448
2022-02-18       683  0.060009
2023-02-18       682  0.059921
...              ...       ...
2020-03-12       417  0.036638
2020-03-09       415  0.036462
2020-11-17       412  0.036199
2020-07-05       412  0.036199
2020-06-18       411  0.036111

[2016 rows x 2 columns]

==================== SAMPLE_TYPE ====================
              count      pct_%
SAMPLE_TYPE                   
TRAIN        720615  63.3

### Tính chính xác

Tuổi khách hàng

In [23]:
rule_acc_age = df['TUOI'] >= 15

LTV thực tế

In [24]:
rule_acc_ltv = df['LTV'] < 400

Hạn mức – dư nợ

In [25]:
rule_acc_limit = df['AFLIMT_MAX'] >= df['CBAL']

Thu nhập – dư nợ

In [26]:
rule_acc_dti = df['CBAL'] <= df['INCOME'] * 20

CIC – DPD

In [27]:
rule_acc_cic = ~(
    (df['MAX_DPD_12M_OBS'] < 30) & (df['MAX_NHOMNOCIC'] > 1)
) & ~(
    (df['MAX_DPD_12M_OBS'] >= 90) & (df['MAX_NHOMNOCIC'] < 3)
)

Flag & tỷ lệ chính xác

In [28]:
df['FLAG_INACCURATE'] = ~(
    rule_acc_age &
    rule_acc_ltv &
    rule_acc_limit &
    rule_acc_dti &
    rule_acc_cic
)

accuracy_rate = 1 - df['FLAG_INACCURATE'].mean()
accuracy_rate

np.float64(0.565977161363219)

In [29]:
rule_acc_age.sum()

np.int64(1138159)

In [30]:
rule_acc_ltv.sum()

np.int64(1138159)

In [31]:
rule_acc_dti.sum()

np.int64(903249)

In [32]:
rule_acc_cic.sum()

np.int64(810189)

### Tính đồng nhất

CBAL vs cấu trúc kỳ hạn

In [33]:
# rule_cons_balance = (
#     df['CBAL'] ==
#     df['CBAL_SHORTTERM_LOAN']
#     + df['CBAL_MIDTERM_LOAN']
#     + df['CBAL_LONGTERM_LOAN']
# )

# df.loc[~rule_cons_balance].shape

Deposit breakdown

In [34]:
rule_cons_deposit = (
    abs(
        df['N_AVG_DEPOSIT_12M'] -
        (df['N_AVG_DD_12M'] + df['N_AVG_CD_12M'])
    ) <= 1
)
df.loc[~rule_cons_deposit].shape

(0, 57)

DPD logic

In [35]:
rule_cons_dpd = df['MAX_DPD_12M_OBS'] <= df['MAX_DPD_12M']

df.loc[~rule_cons_dpd].shape


(0, 57)

Flag tổng hợp consistency

In [36]:
df['FLAG_INCONSISTENT'] = ~(
    # rule_cons_balance &
    rule_cons_deposit &
    rule_cons_dpd
)

BẢNG TỔNG HỢP (RẤT NÊN CÓ)

In [37]:
quality_summary = pd.DataFrame({
    'Metric': [
        'Completeness',
        'Uniqueness',
        'Timeliness',
        'Validity',
        'Accuracy',
        'Consistency'
    ],
    'Issue_Rate_%': [
        df['FLAG_HIGH_MISSING'].mean() * 100,
        dup_full.mean() * 100,
        leak_check.shape[0] / len(df) * 100,
        df['FLAG_INVALID'].mean() * 100,
        df['FLAG_INACCURATE'].mean() * 100,
        df['FLAG_INCONSISTENT'].mean() * 100
    ]
})

quality_summary

,Metric,Issue_Rate_%
0,Completeness,0.000000
1,Uniqueness,0.000000
2,Timeliness,0.000000
3,Validity,0.000000
4,Accuracy,43.402284
5,Consistency,0.000000


In [38]:
df['FLAG_AGE_INVALID'] = (~rule_acc_age).astype(int)
df['FLAG_LTV_HIGH']    = (~rule_acc_ltv).astype(int)
df['FLAG_DTI_HIGH']    = (~rule_acc_dti).astype(int)
df['FLAG_CIC_LOW']     = (~rule_acc_cic).astype(int)
# df['FLAG_INCONSISTENT_DEPOSIT'] = (~rule_cons_deposit).astype(int)

In [39]:
import numpy as np

In [40]:
flag_cols = [c for c in df.columns if c.startswith('FLAG_')]

dq_flag_impact = []

for c in flag_cols:
    tmp = df.groupby(c)['BAD_NEXT_12M'].mean()
    dq_flag_impact.append({
        'FLAG': c,
        'BAD_RATE_0': tmp.get(0, np.nan),
        'BAD_RATE_1': tmp.get(1, np.nan),
        'LIFT': tmp.get(1, np.nan) / tmp.get(0, np.nan)
    })

dq_flag_impact = pd.DataFrame(dq_flag_impact).sort_values('LIFT', ascending=False)
dq_flag_impact

,FLAG,BAD_RATE_0,BAD_RATE_1,LIFT
8,FLAG_DTI_HIGH,0.079144,0.103848,1.312139
4,FLAG_INACCURATE,0.097212,0.067332,0.692631
0,FLAG_SALARY_ACC,0.127272,0.080144,0.629709
9,FLAG_CIC_LOW,0.104162,0.035037,0.336367
1,FLAG_DEPOSIT,0.207950,0.058611,0.281852
2,FLAG_HIGH_MISSING,0.084243,NaN,NaN
3,FLAG_INVALID,0.084243,NaN,NaN
5,FLAG_INCONSISTENT,0.084243,NaN,NaN
6,FLAG_AGE_INVALID,0.084243,NaN,NaN
7,FLAG_LTV_HIGH,0.084243,NaN,NaN


In [41]:
for f in ['FLAG_HIGH_MISSING', 'FLAG_INCONSISTENT']:
    print(f, df[f].value_counts(dropna=False))


FLAG_HIGH_MISSING FLAG_HIGH_MISSING
0    1138159
Name: count, dtype: int64
FLAG_INCONSISTENT FLAG_INCONSISTENT
False    1138159
Name: count, dtype: int64


In [42]:
df.shape

(1138159, 62)

In [43]:
df.drop(['FLAG_DTI_HIGH', 'FLAG_INACCURATE', 'FLAG_CIC_LOW', 'FLAG_HIGH_MISSING', 'FLAG_INCONSISTENT', 'ROW_MISSING_PCT'], axis=1, inplace=True)

In [44]:
df.columns

Index(['SOCIF', 'C_GIOITINH', 'TRINHDO', 'TTHONNHAN', 'SOHUUNHA',
       'NHANVIENBIDV', 'INHERENT_RISK', 'REF_MONTH', 'REF_DAY', 'year',
       'BASE_AUM', 'CURRENT_RISK', 'TUOI', 'SNAPSHOT_DATE', 'INCOME', 'CBAL',
       'CBALORG', 'AFLIMT_MAX', 'AFLIMT_MIN', 'AFLIMT_AVG', 'CBAL_AVG',
       'CBAL_MAX', 'CBAL_MIN', 'COLLATERAL_VALUE', 'LTV', 'N_AVG_DEPOSIT_12M',
       'N_AVG_DEPOSIT_6M', 'N_AVG_DD_12M', 'N_AVG_CD_12M', 'FLAG_SALARY_ACC',
       'FLAG_DEPOSIT', 'CBAL_SHORTTERM_LOAN', 'CBAL_LONGTERM_LOAN',
       'HAS_SHORTTERM_LOAN', 'HAS_LONGTERM_LOAN', 'DURATION_MAX',
       'REMAINING_DURATION_MAX', 'TIME_TO_OP_MAX', 'RATE_AVG', 'PURCOD_MAX',
       'PURCOD_MIN', 'MAX_DPD_12M', 'MAX_DPD_12M_OBS', 'AVG_OD_DPD_12M',
       'SUM_ALL_OD_12M', 'BAD_CURRENT', 'XULYNO', 'MAX_NHOMNOCIC',
       'N_AVG_OVERDUE_CBAL_12M', 'CBAL_TO_INC_12MON', 'REAL_GDP_GROWTH_12M',
       'BAD_NEXT_12M', 'SAMPLE_TYPE', 'FLAG_INVALID', 'FLAG_AGE_INVALID',
       'FLAG_LTV_HIGH'],
      dtype='object')

In [45]:
df_train = df[df['SAMPLE_TYPE'] == 'TRAIN']
df_train_y = df_train['BAD_NEXT_12M']
df_OOS = df[df['SAMPLE_TYPE'] == 'OOS']
df_OOS_y = df_OOS['BAD_NEXT_12M']
df_OOT = df[df['SAMPLE_TYPE'] == 'OOT']
df_OOT_y = df_OOT['BAD_NEXT_12M']

In [46]:
df_train.drop(['SAMPLE_TYPE','BAD_NEXT_12M'], axis=1, inplace=True)
df_OOS.drop(['SAMPLE_TYPE', 'BAD_NEXT_12M'], axis=1, inplace=True)
df_OOT.drop(['SAMPLE_TYPE', 'BAD_NEXT_12M'], axis=1, inplace=True)

In [47]:
df_train.columns

Index(['SOCIF', 'C_GIOITINH', 'TRINHDO', 'TTHONNHAN', 'SOHUUNHA',
       'NHANVIENBIDV', 'INHERENT_RISK', 'REF_MONTH', 'REF_DAY', 'year',
       'BASE_AUM', 'CURRENT_RISK', 'TUOI', 'SNAPSHOT_DATE', 'INCOME', 'CBAL',
       'CBALORG', 'AFLIMT_MAX', 'AFLIMT_MIN', 'AFLIMT_AVG', 'CBAL_AVG',
       'CBAL_MAX', 'CBAL_MIN', 'COLLATERAL_VALUE', 'LTV', 'N_AVG_DEPOSIT_12M',
       'N_AVG_DEPOSIT_6M', 'N_AVG_DD_12M', 'N_AVG_CD_12M', 'FLAG_SALARY_ACC',
       'FLAG_DEPOSIT', 'CBAL_SHORTTERM_LOAN', 'CBAL_LONGTERM_LOAN',
       'HAS_SHORTTERM_LOAN', 'HAS_LONGTERM_LOAN', 'DURATION_MAX',
       'REMAINING_DURATION_MAX', 'TIME_TO_OP_MAX', 'RATE_AVG', 'PURCOD_MAX',
       'PURCOD_MIN', 'MAX_DPD_12M', 'MAX_DPD_12M_OBS', 'AVG_OD_DPD_12M',
       'SUM_ALL_OD_12M', 'BAD_CURRENT', 'XULYNO', 'MAX_NHOMNOCIC',
       'N_AVG_OVERDUE_CBAL_12M', 'CBAL_TO_INC_12MON', 'REAL_GDP_GROWTH_12M',
       'FLAG_INVALID', 'FLAG_AGE_INVALID', 'FLAG_LTV_HIGH'],
      dtype='object')